# prescaledFastAdj kernel evaluations in Python with `prescaledfastadj`

### Problem setting

For a given $d$-dimensional *point cloud* of $n$ points $x_i \in \mathbb{R}^d$, $i=1,\ldots,n$, the associated adjacency matrix has the form
$$
A_{\text{gauss}} = (a_{ij})_{i,j=1}^n \in \mathbb{R}^{n \times n}, \qquad A_{ij} = \exp\left(\frac{-\|\mathbf{x}_i - \mathbf{x}_j\|_2^2}{c^2}\right)
$$
$$
A_{\text{der}} = (a_{ij})_{i,j=1}^n \in \mathbb{R}^{n \times n}, \qquad A_{ij} = \frac{\|\mathbf{x}_i - \mathbf{x}_j\|_2^2}{c^2} \exp\left(\frac{-\|\mathbf{x}_i - \mathbf{x}_j\|_2^2}{c^2}\right)
$$
$$
A_{\text{matern}} = (a_{ij})_{i,j=1}^n \in \mathbb{R}^{n \times n}, \qquad A_{ij} = \exp\left(\frac{-\|\mathbf{x}_i - \mathbf{x}_j\|_2}{c}\right)
$$
for the different kernel designs, where $c$ is a shape parameter. 

We use a fast summation scheme based on the Non-equispaced Fast Fourier Transformation (NFFT) to approximate this matrix by a similar deterministic linear operator that allows cheaper computations.
The quality of the approximation depends on a handful of parameters. 
If these are kept fixed, the computational cost of a single matrix-vector product depends linearly on $n$ but expontially on $d$, as opposed to the naive computations depending quadratically on $n$ and linearly on $d$.
For that reason, this software is targeted at the case of very large $n$ and small $d=2,3,4$.

In our Python code, the data points are stored in the rows of a numpy array `X` of shape `(n,d)`:

In [1]:
import numpy as np

n = 10000
d = 6
# define feature windows
wind = [[0,1,2],[3,4,5]]

points = np.random.randn(n, d)

### Creating the adjacency matrix object

The most convenient way to set up computations with the adjacency matrix is the `AdjacencyMatrix` class. Note that the kernels are of a slightly different design in our paper and we therefore have to add some prefactors.

In [6]:
import prescaledfastadj

# define length-scale parameter sigma and signal variance parameter sf
sigma = 1.0
sf = np.sqrt(1/len(wind))
adj_gauss = [prescaledfastadj.AdjacencyMatrix(points[:,wind[i]], np.sqrt(2)*sigma, kernel=1, setup='default', diagonal=1.0) for i in range(len(wind))]
adj_der = [prescaledfastadj.AdjacencyMatrix(points[:,wind[i]], np.sqrt(2)*sigma, kernel=2, setup='default', diagonal=0.0) for i in range(len(wind))]
adj_matern = [prescaledfastadj.AdjacencyMatrix(points[:,wind[i]], sigma, kernel=3, setup='default', diagonal=1.0) for i in range(len(wind))]

* The `points` array and the Gaussian shape parameter `sigma` are required positional arguments.
* The `setup` argument may be one of the string constants `'default'`, `'fine'`, or `'rough'`, which load setup presets for the parameters of the NFFT fastsum method.
* For finer control over the parameters, pass `setup=fastadj.AccuracySetup(N,p,m,eps,tol)`, where
  - `N` is the NFFT expansion degree (called $n$ in the NFFT),
  - `p` is a smoothness parameter,
  - `m` is a window cutoff parameter,
  - `eps` is the outer boundary width (called $\varepsilon_B$ in the NFFT, while $\varepsilon_I$ is always zero),
  - `tol` is the tolerance to be used for future eigenvalue computations.
* The constructor has an optional fourth argument, `diagonal`, which can be set to manipulate the scalar value on the diagonal of the adjacency matrix. The default is 0. It can also be changed later on via `adj.diagonal`.

### Approximate matrix-vector products

Use `adj.apply` to compute the approximate results of the product with a vector, i.e., the image of that vector under the operator $A$. The vector must be passed as a numpy array of shape `(n,)`, same as the return value. For example, the vector of node degrees can be approximated via the following code:

In [7]:
degrees_gauss = [adj_gauss[i].apply(np.ones(n)) for i in range(len(wind))]
degrees_gauss = (sf**2) * np.sum(degrees_gauss, axis=0)
print("Avg/min/max degree gauss: {:.2f}, {:.2f}, {:.2f}".format(degrees_gauss.mean(), degrees_gauss.min(), degrees_gauss.max()))

degrees_der = [(2/sigma)*adj_der[i].apply(np.ones(n)) for i in range(len(wind))]
degrees_der = (sf**2) * np.sum(degrees_der, axis=0)
print("Avg/min/max degree der: {:.2f}, {:.2f}, {:.2f}".format(degrees_der.mean(), degrees_der.min(), degrees_der.max()))

degrees_matern = [adj_matern[i].apply(np.ones(n)) for i in range(len(wind))]
degrees_matern = (sf**2) * np.sum(degrees_matern, axis=0)
print("Avg/min/max degree matern: {:.2f}, {:.2f}, {:.2f}".format(degrees_matern.mean(), degrees_matern.min(), degrees_matern.max()))

Avg/min/max degree gauss: 1946.95, 172.03, 3435.02
Avg/min/max degree der: 3874.90, 792.53, 5244.73
Avg/min/max degree matern: 907.84, 32.09, 1854.79
